In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt
import numpy as np

import tqdm


config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)

import sys
import os
file_path = os.path.abspath(os.getcwd())
os.chdir(file_path)

import Generated_CNN
from Generated_CNN import *
from load_data import *

model = 3
training_count = 1500
freeze = True
n_count = 50000 #all
kernel_size_0 = 4

stride = 1
if model == 2:
    stride = 1
elif model == 3:
    stride = 2
activation = "sigmoid"
l0 = None
l1 = None
class_count = None
initial_channel = 0
image_size = 0

def load_exp_data(dataset):
    train_images, train_labels, test_images, test_labels = load_data(dataset, grayscale = False, resize = False)
    global class_count
    class_count = len(np.unique(train_labels))
    global initial_channel
    initial_channel = train_images[0].shape[-1]
    global image_size
    image_size = train_images.shape[1]
    
    return train_images, train_labels, test_images, test_labels


def generate_result_name(l0, l1, dataset, training_count, n_count, freeze):
    name = "results/"+dataset + "_"
    name += str(training_count) + "_"
    name += str(n_count) + "_"
    name += str(freeze) + "_"
    if l0 != None:
        name += "l0_"+str(l0.filters) + "_"
    if l1 != None:
        name += "l1_"+str(l1.filters) + "_"
    if model == 3:
        name += "model3_"
    return name


def execute_exp(dataset, model = 1):
    train_images, train_labels, test_images, test_labels = load_exp_data(dataset)
    global training_count, n_count, freeze, l0, l1, class_count, initial_channel, image_size
    
    new_model = extensible_CNN_layer_multi_module_3D(kernel_size = (kernel_size_0,kernel_size_0,initial_channel), stride = stride, activation = activation)

    images_CNN_train = train_images[:training_count].reshape(training_count,1, image_size, image_size, initial_channel, 1)
    epochs = 30
    for i in tqdm.tqdm(range(epochs)):
        new_model, max_inactive_ratio = generate_model_on_images(images_CNN_train, new_model, images_x_0_9, n = 7)
        if max_inactive_ratio < 0.1:
            break
        print("max_inactive_ratio: ", max_inactive_ratio)

    l0 = new_model.get_aggregated_conv_2D(name = "conv_l0_generated")
    
    #obtain the fms for the first layer
    fms_l0 = np.asarray(l0.call(np.asarray(images_CNN_train).astype(np.float32).reshape(training_count, image_size, image_size, initial_channel)))
    
    if model == 2:
        fms_l0 = keras.layers.MaxPooling2D((2, 2))(fms_l0).numpy()
    
    fms_l0 = fms_l0.reshape(training_count, 1 , fms_l0.shape[1], fms_l0.shape[2], fms_l0.shape[3], 1)
    
    new_model_l1 = extensible_CNN_layer_multi_module_3D(kernel_size = (kernel_size_0,kernel_size_0,l0.filters), stride = stride, activation = activation)
    
    for i in tqdm.tqdm(range(epochs)):
        new_model_l1, max_inactive_ratio = generate_model_on_images(fms_l0, new_model_l1, images_x_0_9, n = 10)
        if max_inactive_ratio < 0.1:
            break
        print("max_inactive_ratio: ", max_inactive_ratio)
        
    l1 = new_model_l1.get_aggregated_conv_2D(name = "conv_l1_generated")
    if model == 2:
        model_integration = keras.Sequential([
            l0,
            keras.layers.MaxPooling2D((2, 2)),
            l1,
            #keras.layers.BatchNormalization(),
            #keras.layers.Conv2D(64, (3, 3), strides=1, activation='gelu'),
            #keras.layers.MaxPooling2D((2, 2)),
            #keras.layers.BatchNormalization(),
            #keras.layers.Conv2D(128, (4, 4), strides=1, activation='sigmoid'),
            #dense layers

            keras.layers.MaxPool2D((2, 2)),
            keras.layers.Flatten(),

            keras.layers.Dense(128, activation='sigmoid', name = "dense_integration_1"),
            keras.layers.Dense(class_count, activation='softmax', name = "dense_integration_2")
        ])
    elif model == 3:
        #remove maxpooling
        model_integration = keras.Sequential([
            l0,
            l1,
            keras.layers.Flatten(),

            keras.layers.Dense(128, activation='sigmoid', name = "dense_integration_1"),
            keras.layers.Dense(class_count, activation='softmax', name = "dense_integration_2")
        ])
    model_integration.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    l0.trainable = not freeze
    l1.trainable = not freeze
    
    history_integration = model_integration.fit(train_images[0:n_count], train_labels[0:n_count], epochs=40, validation_data=(test_images, test_labels))
    
    
    #make a comparison with the original model
    
    l0_ord = keras.layers.Conv2D(l0.filters, (kernel_size_0, kernel_size_0), strides=stride, activation=activation, input_shape=(image_size, image_size, initial_channel), name = "conv_l0_original")
    l1_ord = keras.layers.Conv2D(l1.filters, (kernel_size_0, kernel_size_0), strides=stride, activation=activation, name = "conv_l1_original")
    
    if model == 2:
        model_ord = keras.Sequential([
            l0_ord,
            keras.layers.MaxPooling2D((2, 2)),
            l1_ord,

            keras.layers.MaxPool2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation='sigmoid', name="dense_ord_1"),
            keras.layers.Dense(class_count, activation='softmax', name="dense_ord_2")

        ])
    elif model == 3:
        model_ord = keras.Sequential([
            l0_ord,
            l1_ord,
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation='sigmoid', name="dense_ord_1"),
            keras.layers.Dense(class_count, activation='softmax', name="dense_ord_2")

        ])
    
    model_ord.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    print("Images used for training: ", n_count)
    history_ord = model_ord.fit(train_images[0:n_count], train_labels[0:n_count], epochs=40, validation_data=(test_images, test_labels))
    import pickle
    folder_name = generate_result_name(l0, l1, dataset, training_count, n_count, freeze)
    with open(folder_name + "history_integration", 'wb') as file_pi:
        pickle.dump(history_integration.history, file_pi)
    
    with open(folder_name + "history_ord", 'wb') as file_pi:
        pickle.dump(history_ord.history, file_pi)
        
    print("integ_max_val_acc: ", max(history_integration.history['val_accuracy']))
    print("ord_max_val_acc: ", max(history_ord.history['val_accuracy']))
    

#"mnist", "fashion_mnist",  already tested
datasets_to_test = ["cifar10"]

for dataset in datasets_to_test:
    execute_exp(dataset)



  0%|          | 0/30 [00:00<?, ?it/s]


image  1238
image shape (32, 32, 3, 1)
new filter initialized, id =  0
result 0 0.0
max abs 0.0

combined training v3
inactive ratio  1.0
selected point 5 3 0

Updating the sample space
new filter initialized, id =  1
result 0 1.0
max abs 0.141853372315554
sum of matrix_1 =  6.661338147750939e-16
char sum: 6.661338147750939e-16

combined training v3
inactive ratio after adding filter 0.6977777777777778

image  1382
image shape (32, 32, 3, 1)
inactive ratio  1.0
selected point 12 13 0

Updating the sample space
new filter initialized, id =  2
std is too small, increase std to a higher value
0.41087145439506934 [[[[[-0.30337488]
    [ 0.13348622]
    [ 0.22011126]]

   [[-0.48082344]
    [ 0.07968751]
    [ 0.13348622]]

   [[-0.48082344]
    [ 0.06904934]
    [ 0.15871386]]

   [[-0.69356565]
    [ 0.05348301]
    [ 0.25628102]]]


  [[[-0.547816  ]
    [ 0.13348622]
    [ 0.25628102]]

   [[-0.35860289]
    [ 0.18758888]
    [ 0.22011126]]

   [[-0.77232274]
    [ 0.05871513]
    [ 0.

100%|██████████| 1500/1500 [00:20<00:00, 74.35it/s]


inactive ratios mean 0.34089185185185183 [0.057777777777777775, 0.27111111111111114, 0.4088888888888889, 0.5244444444444445, 0.17777777777777778, 0.2, 0.06666666666666667, 0.14222222222222222, 0.017777777777777778, 0.6088888888888889, 0.8888888888888888, 0.35555555555555557, 0.2, 0.5511111111111111, 0.32, 0.1511111111111111, 0.37333333333333335, 0.5733333333333334, 0.64, 0.24888888888888888, 0.4, 0.1288888888888889, 0.7555555555555555, 0.36444444444444446, 0.48444444444444446, 0.39111111111111113, 0.4533333333333333, 0.5822222222222222, 0.47555555555555556, 0.4533333333333333, 0.7022222222222222, 0.4266666666666667, 0.3422222222222222, 0.43555555555555553, 0.07111111111111111, 0.06666666666666667, 0.4533333333333333, 0.52, 0.2222222222222222, 0.7022222222222222, 0.4488888888888889, 0.017777777777777778, 0.17777777777777778, 0.10222222222222223, 0.38222222222222224, 0.12444444444444444, 0.37777777777777777, 0.017777777777777778, 0.2311111111111111, 0.19111111111111112, 0.186666666666666

  3%|▎         | 1/30 [00:36<17:45, 36.74s/it]

...

inactive ratios mean 0.01190222222222222 [0.0, 0.013333333333333334, 0.0, 0.0044444444444444444, 0.008888888888888889, 0.0, 0.0, 0.0, 0.0, 0.0044444444444444444, 0.23555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0044444444444444444, 0.022222222222222223, 0.022222222222222223, 0.0044444444444444444, 0.0, 0.0, 0.017777777777777778, 0.09333333333333334, 0.008888888888888889, 0.022222222222222223, 0.022222222222222223, 0.03111111111111111, 0.008888888888888889, 0.017777777777777778, 0.013333333333333334, 0.022222222222222223, 0.0, 0.02666666666666667, 0.008888888888888889, 0.0, 0.0, 0.013333333333333334, 0.0, 0.02666666666666667, 0.013333333333333334, 0.0, 0.0, 0.0, 0.013333333333333334, 0.0, 0.0044444444444444444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.022222222222222223, 0.008888888888888889, 0.0, 0.0, 0.0, 0.04888888888888889, 0.0, 0.0, 0.008888888888888889, 0.013333333333333334, 0.0, 0.05333333333333334, 0.0, 0.0, 0.0, 0.0, 0.00888888888888

 43%|████▎     | 13/30 [23:19<43:29, 153.50s/it]

inactive ratio after adding filter 0.4266666666666667
removing filter
sigmoid 79
max_inactive_ratio:  0.4311111111111111


 43%|████▎     | 13/30 [25:12<32:58, 116.38s/it]


KeyboardInterrupt: 